### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import astropy.units as u
from survey_tools import catalog, match
from survey_tools.utility import plot

### Options

In [ ]:
do_save = False
do_save_ascii = False
keep_open = False

plot_mode   = 'default'
plot_options   = plot.get_plot_colours(plot_mode)

### Load UltraVISTA Catalog (v4.1, 2015-08-20)

In [ ]:
catalog_params_UVISTA = catalog.get_params('UVISTA', 'COSMOS', 'K')
catalog_UVISTA = catalog.read(catalog_params_UVISTA)
catalog_region_UVISTA = catalog.get_catalog_region(catalog_UVISTA)
print(f"{catalog_UVISTA.catalog}: {catalog_UVISTA.count} sources")
print('\nUltraVISTA Region:')
print(catalog_region_UVISTA)

### Cross-Match zCOSMOS Bright (DR3, 2016-01-19)

In [ ]:
catalog_params_ZCB = catalog.get_params('ZCOSMOS-BRIGHT')
catalog_ZCB = catalog.read(catalog_params_ZCB)
print(f"{catalog_ZCB.catalog}: {catalog_ZCB.count} sources")

In [ ]:
matches_ZCB = match.cross_match_catalogs_by_id_ra_dec(catalog_UVISTA, catalog_ZCB, id_field1='z_spec_id')
print(f"Number of Matches: {matches_ZCB.count}")

In [ ]:
match.append_cross_matched_data(matches_ZCB, catalog_UVISTA, catalog_ZCB)

In [ ]:
value_filter       = (catalog_UVISTA.idmap['id']  < 1e7) & (catalog_UVISTA.idmap['ZCB'] >= 0)
other_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['ZCB'] >= 0) & (catalog_UVISTA.idmap['ZCB_dist'] > 0.0)
multi_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['ZCB'] >= 0) & (catalog_UVISTA.idmap['ZCB_dist'] == 0.0) & (catalog_UVISTA.idmap['ZCB_num'] > 1)
dup_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['ZCB'] >= 0) & (catalog_UVISTA.idmap['ZCB_dist'] == 0.0) & (catalog_UVISTA.idmap['ZCB_num'] == 1)
non_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['ZCB'] >= 0) & (catalog_UVISTA.idmap['ZCB_dist'] == 0.0) & (catalog_UVISTA.idmap['ZCB_num'] == 0)

num_matches = np.sum(value_filter)
num_other_matches = np.sum(other_match_filter)
num_multi_matches = np.sum(multi_match_filter)
num_dup_matches = np.sum(dup_match_filter)
num_non_matches = np.sum(non_match_filter)
print(f"Number of {catalog_ZCB.name} Redshifts:")
print(f"    Matches      : {num_matches} (diff={matches_ZCB.count - num_matches - num_other_matches})")
print(f"  + Other matches: {num_other_matches}")
print(f"  + Dup-matches  : {num_dup_matches}")
print(f"  + Multi-matches: {num_multi_matches}")
print(f"  + Non-matches  : {num_non_matches}")
print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_ZCB.original_count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

zlim_deep = [0, 3]
zlim = [0, 2]

value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_ZCB'] > 0.0) & (catalog_UVISTA.redshift['z_ZCB_flag'] < 8)
fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_ZCB.catalog} vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_ZCB'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
ax.set_xlabel(f"$z_{{peak}}$ [{catalog_UVISTA.catalog}]")
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_ZCB.catalog}]")
ax.set_xlim(zlim_deep)
ax.set_ylim(zlim_deep)

fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_ZCB.catalog}")
value_filter = (catalog_UVISTA.redshift['z_spec'] > 0.0) & (catalog_UVISTA.redshift['z_ZCB'] > 0.0) & (catalog_UVISTA.redshift['z_ZCB_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_spec'][value_filter], catalog_UVISTA.redshift['z_ZCB'][value_filter], label=f"UVISTA ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
ax.set_xlabel('$z_{spec}$ [Other]')
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_ZCB.catalog}]")
ax.set_xlim(zlim)
ax.set_ylim(zlim)
ax.legend(loc='lower right')

del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_ZCB' in locals():
    catalog.close(catalog_ZCB)
    del catalog_ZCB, matches_ZCB, catalog_params_ZCB

### Cross-Match zCOSMOS Deep (unreleased, between 2009-2016?)

In [ ]:
catalog_params_ZCD = catalog.get_params('ZCOSMOS-DEEP')
catalog_ZCD = catalog.read(catalog_params_ZCD)
print(f"{catalog_ZCD.catalog}: {catalog_ZCD.count} sources")

In [ ]:
matches_ZCD = match.cross_match_catalogs_by_ra_dec(catalog_UVISTA, catalog_ZCD)
print(f"Number of Matches: {matches_ZCD.count}")

In [ ]:
match.append_cross_matched_data(matches_ZCD, catalog_UVISTA, catalog_ZCD)

In [ ]:
value_filter   = (catalog_UVISTA.idmap['id'] < 1e7) & (catalog_UVISTA.idmap['ZCD'] >= 0)
other_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['ZCD'] >= 0) & (catalog_UVISTA.idmap['ZCD_dist'] > 0.0)
multi_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['ZCD'] >= 0) & (catalog_UVISTA.idmap['ZCD_dist'] == 0.0) & (catalog_UVISTA.idmap['ZCD_num'] > 1)
dup_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['ZCD'] >= 0) & (catalog_UVISTA.idmap['ZCD_dist'] == 0.0) & (catalog_UVISTA.idmap['ZCD_num'] == 1)
non_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['ZCD'] >= 0) & (catalog_UVISTA.idmap['ZCD_dist'] == 0.0) & (catalog_UVISTA.idmap['ZCD_num'] == 0)

num_matches = np.sum(value_filter)
num_other_matches = np.sum(other_match_filter)
num_multi_matches = np.sum(multi_match_filter)
num_dup_matches = np.sum(dup_match_filter)
num_non_matches = np.sum(non_match_filter)
print(f"Number of {catalog_ZCD.name} Redshifts:")
print(f"    Matches      : {num_matches} (diff={matches_ZCD.count - num_matches - num_other_matches})")
print(f"  + Other matches: {num_other_matches}")
print(f"  + Dup-matches  : {num_dup_matches}")
print(f"  + Multi-matches: {num_multi_matches}")
print(f"  + Non-matches  : {num_non_matches}")
print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_ZCD.original_count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

zlim_deep = [0, 6]
zlim = [0, 3.5]

value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_ZCD'] > 0.0) & (catalog_UVISTA.redshift['z_ZCD_flag'] < 8)
fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_ZCD.catalog} vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_ZCD'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
ax.set_xlabel(f"$z_{{peak}}$ [{catalog_UVISTA.catalog}]")
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_ZCD.catalog}]")
ax.set_xlim(zlim_deep)
ax.set_ylim(zlim_deep)

fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_ZCD.catalog}")
value_filter = (catalog_UVISTA.redshift['z_ZCB'] > 0.0) & (catalog_UVISTA.redshift['z_ZCB_flag'] < 8) & (catalog_UVISTA.redshift['z_ZCD'] > 0.0) & (catalog_UVISTA.redshift['z_ZCD_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCB'][value_filter], catalog_UVISTA.redshift['z_ZCD'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
ax.set_xlabel('$z_{spec}$ [Other]')
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_ZCD.catalog}]")
ax.set_xlim(zlim)
ax.set_ylim(zlim)
ax.legend(loc='lower right')

del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_ZCD' in locals():
    catalog.close(catalog_ZCD)
    del catalog_ZCD, matches_ZCD, catalog_params_ZCD

### Cross-Match 3D-HST (v4.1, 2014-09-03)

In [ ]:
catalog_params_3DHST = catalog.get_params('3D-HST', 'COSMOS', 'F160W')
catalog_3DHST = catalog.read(catalog_params_3DHST)
print(f"3D-HST: {catalog_3DHST.count} sources")

In [ ]:
matches_3DHST = match.cross_match_catalogs_by_ra_dec(catalog_UVISTA, catalog_3DHST)
print(f"Number of Matches: {matches_3DHST.count}")

In [ ]:
match.append_cross_matched_data(matches_3DHST, catalog_UVISTA, catalog_3DHST)

In [ ]:
value_filter   = (catalog_UVISTA.idmap['id'] < 1e7) & (catalog_UVISTA.idmap['3DHST'] >= 0)
other_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['3DHST'] >= 0) & (catalog_UVISTA.idmap['3DHST_dist'] > 0.0)
multi_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['3DHST'] >= 0) & (catalog_UVISTA.idmap['3DHST_dist'] == 0.0) & (catalog_UVISTA.idmap['3DHST_num'] > 1)
dup_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['3DHST'] >= 0) & (catalog_UVISTA.idmap['3DHST_dist'] == 0.0) & (catalog_UVISTA.idmap['3DHST_num'] == 1)
non_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['3DHST'] >= 0) & (catalog_UVISTA.idmap['3DHST_dist'] == 0.0) & (catalog_UVISTA.idmap['3DHST_num'] == 0)

num_matches = np.sum(value_filter)
num_other_matches = np.sum(other_match_filter)
num_multi_matches = np.sum(multi_match_filter)
num_dup_matches = np.sum(dup_match_filter)
num_non_matches = np.sum(non_match_filter)
print(f"Number of {catalog_3DHST.name} Redshifts:")
print(f"    Matches      : {num_matches} (diff={matches_3DHST.count - num_matches - num_other_matches})")
print(f"  + Other matches: {num_other_matches}")
print(f"  + Dup matches  : {num_dup_matches}")
print(f"  + Multi matches: {num_multi_matches}")
print(f"  + Non matches  : {num_non_matches}")
print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_3DHST.original_count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

zlim_deep = [0, 5]
zlim = [0, 4]

value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_3DHST'] > 0.0) & (catalog_UVISTA.redshift['z_3DHST_flag'] < 8)
fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_3DHST.catalog} vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_3DHST'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
ax.set_xlabel(f"$z_{{peak}}$ [{catalog_UVISTA.catalog}]")
ax.set_ylabel(f"$z_{{best}}$ [{catalog_3DHST.catalog}]")
ax.set_xlim(zlim_deep)
ax.set_ylim(zlim_deep)

fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_3DHST.catalog}")
value_filter = (catalog_UVISTA.redshift['z_ZCB'] > 0.0) & (catalog_UVISTA.redshift['z_ZCB_flag'] < 8) & (catalog_UVISTA.redshift['z_3DHST'] > 0.0) & (catalog_UVISTA.redshift['z_3DHST_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCB'][value_filter], catalog_UVISTA.redshift['z_3DHST'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
value_filter = (catalog_UVISTA.redshift['z_ZCD'] > 0.0) & (catalog_UVISTA.redshift['z_ZCD_flag'] < 8) & (catalog_UVISTA.redshift['z_3DHST'] > 0.0) & (catalog_UVISTA.redshift['z_3DHST_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCD'][value_filter], catalog_UVISTA.redshift['z_3DHST'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
ax.set_xlabel('$z_{spec}$ [Other]')
ax.set_ylabel(f"$z_{{best}}$ [{catalog_3DHST.catalog}]")
ax.set_xlim(zlim)
ax.set_ylim(zlim)
ax.legend(loc='lower right')

del zlim_deep, zlim, fig, ax

In [ ]:
dz_threshold = 0.01

best_filter = (abs(catalog_UVISTA.redshift['z_peak'] - catalog_UVISTA.redshift['z_3DHST']) < dz_threshold) \
            & (catalog_UVISTA.redshift['z_3DHST_flag'] < 3)

massLim = [6, 12]
value_filter = (catalog_UVISTA.spp['lmass'] > -99) & (catalog_UVISTA.spp['lmass_3DHST'] > -99)
fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_3DHST.catalog} vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}, Best={np.sum(value_filter & best_filter)}")
ax.scatter(catalog_UVISTA.spp['lmass'][value_filter], catalog_UVISTA.spp['lmass_3DHST'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
ax.scatter(catalog_UVISTA.spp['lmass'][value_filter & best_filter], catalog_UVISTA.spp['lmass_3DHST'][value_filter & best_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
ax.plot(massLim, massLim, linestyle='-', color=plot_options.colour7)
ax.set_xlabel(f"log Mass [{catalog_UVISTA.catalog}]")
ax.set_ylabel(f"log Mass [{catalog_3DHST.catalog}]")
ax.set_xlim(massLim)
ax.set_ylim(massLim)

sfrLim = [-5, 5]
value_filter = (catalog_UVISTA.spp['lsfr'] > -99) & (catalog_UVISTA.spp['lsfr_3DHST'] > -99)
fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_3DHST.catalog} vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}, Best={np.sum(value_filter & best_filter)}")
ax.scatter(catalog_UVISTA.spp['lsfr'][value_filter], catalog_UVISTA.spp['lsfr_3DHST'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
ax.scatter(catalog_UVISTA.spp['lsfr'][value_filter & best_filter], catalog_UVISTA.spp['lsfr_3DHST'][value_filter & best_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
ax.plot(sfrLim, sfrLim, linestyle='-', color=plot_options.colour7)
ax.set_xlabel(f"log SFR [{catalog_UVISTA.catalog}]")
ax.set_ylabel(f"log SFR [{catalog_3DHST.catalog}]")
ax.set_xlim(sfrLim)
ax.set_ylim(sfrLim)

avLim = [0, 3]
value_filter = (catalog_UVISTA.spp['Av'] > -99) & (catalog_UVISTA.spp['Av_3DHST'] > -99)
fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_3DHST.catalog} vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}, Best={np.sum(value_filter & best_filter)}")
ax.scatter(catalog_UVISTA.spp['Av'][value_filter], catalog_UVISTA.spp['Av_3DHST'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
ax.scatter(catalog_UVISTA.spp['Av'][value_filter & best_filter], catalog_UVISTA.spp['Av_3DHST'][value_filter & best_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
ax.plot(avLim, avLim, linestyle='-', color=plot_options.colour7)
ax.set_xlabel(f"Av [{catalog_UVISTA.catalog}]")
ax.set_ylabel(f"Av [{catalog_3DHST.catalog}]")
ax.set_xlim(avLim)
ax.set_ylim(avLim)

del dz_threshold, best_filter, value_filter, massLim, sfrLim, avLim, fig, ax

In [ ]:
if not keep_open and 'catalog_3DHST' in locals():
    catalog.close(catalog_3DHST)
    del catalog_3DHST, matches_3DHST, catalog_params_3DHST

### Cross-Match VUDS (DR1, 2015-08-12)

In [ ]:
catalog_params_VUDS = catalog.get_params('VUDS', 'COSMOS')
catalog_VUDS = catalog.read(catalog_params_VUDS)
print(f"{catalog_VUDS.catalog}: {catalog_VUDS.count} sources")

In [ ]:
matches_VUDS = match.cross_match_catalogs_by_ra_dec(catalog_UVISTA, catalog_VUDS)
print(f"Number of Matches: {matches_VUDS.count}")

In [ ]:
match.append_cross_matched_data(matches_VUDS, catalog_UVISTA, catalog_VUDS)

In [ ]:
value_filter   = (catalog_UVISTA.idmap['id']  < 1e7) & (catalog_UVISTA.idmap['VUDS'] >= 0)
other_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['VUDS'] >= 0) & (catalog_UVISTA.idmap['VUDS_dist'] > 0.0)
multi_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['VUDS'] >= 0) & (catalog_UVISTA.idmap['VUDS_dist'] == 0.0) & (catalog_UVISTA.idmap['VUDS_num'] > 1)
dup_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['VUDS'] >= 0) & (catalog_UVISTA.idmap['VUDS_dist'] == 0.0) & (catalog_UVISTA.idmap['VUDS_num'] == 1)
non_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['VUDS'] >= 0) & (catalog_UVISTA.idmap['VUDS_dist'] == 0.0) & (catalog_UVISTA.idmap['VUDS_num'] == 0)

num_matches = np.sum(value_filter)
num_other_matches = np.sum(other_match_filter)
num_multi_matches = np.sum(multi_match_filter)
num_dup_matches = np.sum(dup_match_filter)
num_non_matches = np.sum(non_match_filter)
print(f"Number of {catalog_VUDS.name} Redshifts:")
print(f"    Matches      : {num_matches} (diff={matches_VUDS.count - num_matches - num_other_matches})")
print(f"  + Other matches: {num_other_matches}")
print(f"  + Dup-matches  : {num_dup_matches}")
print(f"  + Multi-matches: {num_multi_matches}")
print(f"  + Non-matches  : {num_non_matches}")
print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_VUDS.original_count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

zlim_deep = [0, 6]
zlim = [0, 6]

value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_VUDS'] > 0.0) & (catalog_UVISTA.redshift['z_VUDS_flag'] < 8)
fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_VUDS.catalog} vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_VUDS'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
ax.set_xlabel(f"$z_{{peak}}$ [{catalog_UVISTA.catalog}]")
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_VUDS.catalog}]")
ax.set_xlim(zlim_deep)
ax.set_ylim(zlim_deep)

fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_VUDS.catalog}")
value_filter = (catalog_UVISTA.redshift['z_ZCB'] > 0.0) & (catalog_UVISTA.redshift['z_ZCB_flag'] < 8) & (catalog_UVISTA.redshift['z_VUDS'] > 0.0) & (catalog_UVISTA.redshift['z_VUDS_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCB'][value_filter], catalog_UVISTA.redshift['z_VUDS'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
value_filter = (catalog_UVISTA.redshift['z_ZCD'] > 0.0) & (catalog_UVISTA.redshift['z_ZCD_flag'] < 8) & (catalog_UVISTA.redshift['z_VUDS'] > 0.0) & (catalog_UVISTA.redshift['z_VUDS_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCD'][value_filter], catalog_UVISTA.redshift['z_VUDS'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
value_filter = (catalog_UVISTA.redshift['z_3DHST'] > 0.0) & (catalog_UVISTA.redshift['z_3DHST_flag'] < 8) & (catalog_UVISTA.redshift['z_VUDS'] > 0.0) & (catalog_UVISTA.redshift['z_VUDS_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_3DHST'][value_filter], catalog_UVISTA.redshift['z_VUDS'][value_filter], label=f"3DHST ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour4)
ax.set_xlabel('$z_{spec}$ [Other]')
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_VUDS.catalog}]")
ax.set_xlim(zlim)
ax.set_ylim(zlim)
ax.legend(loc='lower right')

del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_VUDS' in locals():
    catalog.close(catalog_VUDS)
    del catalog_VUDS, matches_VUDS, catalog_params_VUDS

### Cross-Match Casey DSFG (2017-05-10)

In [ ]:
catalog_params_Casey = catalog.get_params('Casey', 'COSMOS')
catalog_Casey = catalog.read(catalog_params_Casey)
print(f"{catalog_Casey.catalog}: {catalog_Casey.count} sources")

In [ ]:
matches_Casey = match.cross_match_catalogs_by_ra_dec(catalog_UVISTA, catalog_Casey)
print(f"Number of Matches: {matches_Casey.count}")

In [ ]:
match.append_cross_matched_data(matches_Casey, catalog_UVISTA, catalog_Casey)

In [ ]:
value_filter   = (catalog_UVISTA.idmap['id']  < 1e7) & (catalog_UVISTA.idmap['Casey'] >= 0)
other_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['Casey'] >= 0) & (catalog_UVISTA.idmap['Casey_dist'] > 0.0)
multi_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['Casey'] >= 0) & (catalog_UVISTA.idmap['Casey_dist'] == 0.0) & (catalog_UVISTA.idmap['Casey_num'] > 1)
dup_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['Casey'] >= 0) & (catalog_UVISTA.idmap['Casey_dist'] == 0.0) & (catalog_UVISTA.idmap['Casey_num'] == 1)
non_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['Casey'] >= 0) & (catalog_UVISTA.idmap['Casey_dist'] == 0.0) & (catalog_UVISTA.idmap['Casey_num'] == 0)

num_matches = np.sum(value_filter)
num_other_matches = np.sum(other_match_filter)
num_multi_matches = np.sum(multi_match_filter)
num_dup_matches = np.sum(dup_match_filter)
num_non_matches = np.sum(non_match_filter)
print(f"Number of {catalog_Casey.name} Redshifts:")
print(f"    Matches      : {num_matches} (diff={matches_Casey.count - num_matches - num_other_matches})")
print(f"  + Other matches: {num_other_matches}")
print(f"  + Dup-matches  : {num_dup_matches}")
print(f"  + Multi-matches: {num_multi_matches}")
print(f"  + Non-matches  : {num_non_matches}")
print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_Casey.original_count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

zlim_deep = [0, 3.5]
zlim = [0, 1.5]

value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_Casey'] > 0.0) & (catalog_UVISTA.redshift['z_Casey_flag'] < 8)
fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_Casey.catalog} vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_Casey'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
ax.set_xlabel(f"$z_{{peak}}$ [{catalog_UVISTA.catalog}]")
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_Casey.catalog}]")
ax.set_xlim(zlim_deep)
ax.set_ylim(zlim_deep)

fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_Casey.catalog}")
value_filter = (catalog_UVISTA.redshift['z_ZCB'] > 0.0) & (catalog_UVISTA.redshift['z_ZCB_flag'] < 8) & (catalog_UVISTA.redshift['z_Casey'] > 0.0) & (catalog_UVISTA.redshift['z_Casey_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCB'][value_filter], catalog_UVISTA.redshift['z_Casey'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
value_filter = (catalog_UVISTA.redshift['z_ZCD'] > 0.0) & (catalog_UVISTA.redshift['z_ZCD_flag'] < 8) & (catalog_UVISTA.redshift['z_Casey'] > 0.0) & (catalog_UVISTA.redshift['z_Casey_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCD'][value_filter], catalog_UVISTA.redshift['z_Casey'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
value_filter = (catalog_UVISTA.redshift['z_3DHST'] > 0.0) & (catalog_UVISTA.redshift['z_3DHST_flag'] < 8) & (catalog_UVISTA.redshift['z_Casey'] > 0.0) & (catalog_UVISTA.redshift['z_Casey_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_3DHST'][value_filter], catalog_UVISTA.redshift['z_Casey'][value_filter], label=f"3DHST ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour4)
value_filter = (catalog_UVISTA.redshift['z_VUDS'] > 0.0) & (catalog_UVISTA.redshift['z_VUDS_flag'] < 8) & (catalog_UVISTA.redshift['z_Casey'] > 0.0) & (catalog_UVISTA.redshift['z_Casey_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_VUDS'][value_filter], catalog_UVISTA.redshift['z_Casey'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
ax.set_xlabel('$z_{spec}$ [Other]')
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_Casey.catalog}]")
ax.set_xlim(zlim)
ax.set_ylim(zlim)
ax.legend(loc='lower right')

del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_Casey' in locals():
    catalog.close(catalog_Casey)
    del catalog_Casey, matches_Casey, catalog_params_Casey

### Cross-Match DEIMOS 10K (2017-12-27)

In [ ]:
catalog_params_DEIMOS = catalog.get_params('DEIMOS')
catalog_DEIMOS = catalog.read(catalog_params_DEIMOS)
print(f"{catalog_DEIMOS.catalog}: {catalog_DEIMOS.count} sources")

In [ ]:
matches_DEIMOS = match.cross_match_catalogs_by_ra_dec(catalog_UVISTA, catalog_DEIMOS)
print(f"Number of Matches: {matches_DEIMOS.count}")

In [ ]:
match.append_cross_matched_data(matches_DEIMOS, catalog_UVISTA, catalog_DEIMOS)

In [ ]:
value_filter   = (catalog_UVISTA.idmap['id'] < 1e7) & (catalog_UVISTA.idmap['DEIMOS'] >= 0)
other_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['DEIMOS'] >= 0) & (catalog_UVISTA.idmap['DEIMOS_dist'] > 0.0)
multi_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['DEIMOS'] >= 0) & (catalog_UVISTA.idmap['DEIMOS_dist'] == 0.0) & (catalog_UVISTA.idmap['DEIMOS_num'] > 1)
dup_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['DEIMOS'] >= 0) & (catalog_UVISTA.idmap['DEIMOS_dist'] == 0.0) & (catalog_UVISTA.idmap['DEIMOS_num'] == 1)
non_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['DEIMOS'] >= 0) & (catalog_UVISTA.idmap['DEIMOS_dist'] == 0.0) & (catalog_UVISTA.idmap['DEIMOS_num'] == 0)

num_matches = np.sum(value_filter)
num_other_matches = np.sum(other_match_filter)
num_multi_matches = np.sum(multi_match_filter)
num_dup_matches = np.sum(dup_match_filter)
num_non_matches = np.sum(non_match_filter)
print(f"Number of {catalog_DEIMOS.name} Redshifts:")
print(f"    Matches      : {num_matches} (diff={matches_DEIMOS.count - num_matches - num_other_matches})")
print(f"  + Other matches: {num_other_matches}")
print(f"  + Dup-matches  : {num_dup_matches}")
print(f"  + Multi-matches: {num_multi_matches}")
print(f"  + Non-matches  : {num_non_matches}")
print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_DEIMOS.original_count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

zlim_deep = [0, 6]
zlim = [0, 6]

value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_DEIMOS'] > 0.0) & (catalog_UVISTA.redshift['z_DEIMOS_flag'] < 8)
fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_DEIMOS.catalog} vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_DEIMOS'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
ax.set_xlabel(f"$z_{{peak}}$ [{catalog_UVISTA.catalog}]")
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_DEIMOS.catalog}]")
ax.set_xlim(zlim_deep)
ax.set_ylim(zlim_deep)

fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_DEIMOS.catalog}")
value_filter = (catalog_UVISTA.redshift['z_ZCB'] > 0.0) & (catalog_UVISTA.redshift['z_ZCB_flag'] < 8) & (catalog_UVISTA.redshift['z_DEIMOS'] > 0.0) & (catalog_UVISTA.redshift['z_DEIMOS_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCB'][value_filter], catalog_UVISTA.redshift['z_DEIMOS'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
value_filter = (catalog_UVISTA.redshift['z_ZCD'] > 0.0) & (catalog_UVISTA.redshift['z_ZCD_flag'] < 8) & (catalog_UVISTA.redshift['z_DEIMOS'] > 0.0) & (catalog_UVISTA.redshift['z_DEIMOS_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCD'][value_filter], catalog_UVISTA.redshift['z_DEIMOS'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
value_filter = (catalog_UVISTA.redshift['z_3DHST'] > 0.0) & (catalog_UVISTA.redshift['z_3DHST_flag'] < 8) & (catalog_UVISTA.redshift['z_DEIMOS'] > 0.0) & (catalog_UVISTA.redshift['z_DEIMOS_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_3DHST'][value_filter], catalog_UVISTA.redshift['z_DEIMOS'][value_filter], label=f"3DHST ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour4)
value_filter = (catalog_UVISTA.redshift['z_VUDS'] > 0.0) & (catalog_UVISTA.redshift['z_VUDS_flag'] < 8) & (catalog_UVISTA.redshift['z_DEIMOS'] > 0.0) & (catalog_UVISTA.redshift['z_DEIMOS_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_VUDS'][value_filter], catalog_UVISTA.redshift['z_DEIMOS'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
value_filter = (catalog_UVISTA.redshift['z_Casey'] > 0.0) & (catalog_UVISTA.redshift['z_Casey_flag'] < 8) & (catalog_UVISTA.redshift['z_DEIMOS'] > 0.0) & (catalog_UVISTA.redshift['z_DEIMOS_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_Casey'][value_filter], catalog_UVISTA.redshift['z_DEIMOS'][value_filter], label=f"Casey ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
ax.set_xlabel('$z_{spec}$ [Other]')
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_DEIMOS.catalog}]")
ax.set_xlim(zlim)
ax.set_ylim(zlim)
ax.legend(loc='lower right')

del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_DEIMOS' in locals():
    catalog.close(catalog_DEIMOS)
    del catalog_DEIMOS, matches_DEIMOS, catalog_params_DEIMOS

### Cross-Match MOSDEF (Final, 2018-03-11)

In [ ]:
catalog_params_MOSDEF = catalog.get_params('MOSDEF', 'COSMOS')
catalog_MOSDEF = catalog.read(catalog_params_MOSDEF)
print(f"{catalog_MOSDEF.catalog}: {catalog_MOSDEF.count} sources")

In [ ]:
matches_MOSDEF = match.cross_match_catalogs_by_ra_dec(catalog_UVISTA, catalog_MOSDEF)
print(f"Number of Matches: {matches_MOSDEF.count}")

In [ ]:
match.append_cross_matched_data(matches_MOSDEF, catalog_UVISTA, catalog_MOSDEF)

In [ ]:
value_filter   = (catalog_UVISTA.idmap['id'] < 1e7) & (catalog_UVISTA.idmap['MOSDEF'] >= 0)
other_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['MOSDEF'] >= 0) & (catalog_UVISTA.idmap['MOSDEF_dist'] > 0.0)
multi_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['MOSDEF'] >= 0) & (catalog_UVISTA.idmap['MOSDEF_dist'] == 0.0) & (catalog_UVISTA.idmap['MOSDEF_num'] > 1)
dup_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['MOSDEF'] >= 0) & (catalog_UVISTA.idmap['MOSDEF_dist'] == 0.0) & (catalog_UVISTA.idmap['MOSDEF_num'] == 1)
non_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['MOSDEF'] >= 0) & (catalog_UVISTA.idmap['MOSDEF_dist'] == 0.0) & (catalog_UVISTA.idmap['MOSDEF_num'] == 0)

num_matches = np.sum(value_filter)
num_other_matches = np.sum(other_match_filter)
num_multi_matches = np.sum(multi_match_filter)
num_dup_matches = np.sum(dup_match_filter)
num_non_matches = np.sum(non_match_filter)
print(f"Number of {catalog_MOSDEF.name} Redshifts:")
print(f"    Matches      : {num_matches} (diff={matches_MOSDEF.count - num_matches - num_other_matches})")
print(f"  + Other matches: {num_other_matches}")
print(f"  + Dup-matches  : {num_dup_matches}")
print(f"  + Multi-matches: {num_multi_matches}")
print(f"  + Non-matches  : {num_non_matches}")
print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_MOSDEF.original_count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

zlim_deep = [0, 4]
zlim = [0, 4]

value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_MOSDEF'] > 0.0) & (catalog_UVISTA.redshift['z_MOSDEF_flag'] < 8)
fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_MOSDEF.catalog} vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_MOSDEF'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
ax.set_xlabel(f"$z_{{peak}}$ [{catalog_UVISTA.catalog}]")
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_MOSDEF.catalog}]")
ax.set_xlim(zlim_deep)
ax.set_ylim(zlim_deep)

fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_MOSDEF.catalog}")
value_filter = (catalog_UVISTA.redshift['z_ZCB'] > 0.0) & (catalog_UVISTA.redshift['z_ZCB_flag'] < 8) & (catalog_UVISTA.redshift['z_MOSDEF'] > 0.0) & (catalog_UVISTA.redshift['z_MOSDEF_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCB'][value_filter], catalog_UVISTA.redshift['z_MOSDEF'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
value_filter = (catalog_UVISTA.redshift['z_ZCD'] > 0.0) & (catalog_UVISTA.redshift['z_ZCD_flag'] < 8) & (catalog_UVISTA.redshift['z_MOSDEF'] > 0.0) & (catalog_UVISTA.redshift['z_MOSDEF_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCD'][value_filter], catalog_UVISTA.redshift['z_MOSDEF'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
value_filter = (catalog_UVISTA.redshift['z_3DHST'] > 0.0) & (catalog_UVISTA.redshift['z_3DHST_flag'] < 8) & (catalog_UVISTA.redshift['z_MOSDEF'] > 0.0) & (catalog_UVISTA.redshift['z_MOSDEF_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_3DHST'][value_filter], catalog_UVISTA.redshift['z_MOSDEF'][value_filter], label=f"3DHST ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour4)
value_filter = (catalog_UVISTA.redshift['z_VUDS'] > 0.0) & (catalog_UVISTA.redshift['z_VUDS_flag'] < 8) & (catalog_UVISTA.redshift['z_MOSDEF'] > 0.0) & (catalog_UVISTA.redshift['z_MOSDEF_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_VUDS'][value_filter], catalog_UVISTA.redshift['z_MOSDEF'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
value_filter = (catalog_UVISTA.redshift['z_Casey'] > 0.0) & (catalog_UVISTA.redshift['z_Casey_flag'] < 8) & (catalog_UVISTA.redshift['z_MOSDEF'] > 0.0) & (catalog_UVISTA.redshift['z_MOSDEF_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_Casey'][value_filter], catalog_UVISTA.redshift['z_MOSDEF'][value_filter], label=f"Casey ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
value_filter = (catalog_UVISTA.redshift['z_DEIMOS'] > 0.0) & (catalog_UVISTA.redshift['z_DEIMOS_flag'] < 8) & (catalog_UVISTA.redshift['z_MOSDEF'] > 0.0) & (catalog_UVISTA.redshift['z_MOSDEF_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_DEIMOS'][value_filter], catalog_UVISTA.redshift['z_MOSDEF'][value_filter], label=f"DEIMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour5)
ax.set_xlabel('$z_{spec}$ [Other]')
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_MOSDEF.catalog}]")
ax.set_xlim(zlim)
ax.set_ylim(zlim)
ax.legend(loc='lower right')

del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_MOSDEF' in locals():
    catalog.close(catalog_MOSDEF)
    del catalog_MOSDEF, matches_MOSDEF, catalog_params_MOSDEF

### Cross-Match FMOS (v2, 2019-01-25)

In [ ]:
catalog_params_FMOS = catalog.get_params('FMOS')
catalog_FMOS = catalog.read(catalog_params_FMOS)
print(f"{catalog_FMOS.catalog}: {catalog_FMOS.count} sources")

In [ ]:
matches_FMOS = match.cross_match_catalogs_by_ra_dec(catalog_UVISTA, catalog_FMOS)
print(f"Number of Matches: {matches_FMOS.count}")

In [ ]:
match.append_cross_matched_data(matches_FMOS, catalog_UVISTA, catalog_FMOS)

In [ ]:
value_filter   = (catalog_UVISTA.idmap['id'] < 1e7) & (catalog_UVISTA.idmap['FMOS'] >= 0)
other_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['FMOS'] >= 0) & (catalog_UVISTA.idmap['FMOS_dist'] > 0.0)
multi_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['FMOS'] >= 0) & (catalog_UVISTA.idmap['FMOS_dist'] == 0.0) & (catalog_UVISTA.idmap['FMOS_num'] > 1)
dup_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['FMOS'] >= 0) & (catalog_UVISTA.idmap['FMOS_dist'] == 0.0) & (catalog_UVISTA.idmap['FMOS_num'] == 1)
non_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['FMOS'] >= 0) & (catalog_UVISTA.idmap['FMOS_dist'] == 0.0) & (catalog_UVISTA.idmap['FMOS_num'] == 0)

num_matches = np.sum(value_filter)
num_other_matches = np.sum(other_match_filter)
num_multi_matches = np.sum(multi_match_filter)
num_dup_matches = np.sum(dup_match_filter)
num_non_matches = np.sum(non_match_filter)
print(f"Number of {catalog_FMOS.name} Redshifts:")
print(f"    Matches      : {num_matches} (diff={matches_FMOS.count - num_matches - num_other_matches})")
print(f"  + Other matches: {num_other_matches}")
print(f"  + Dup-matches  : {num_dup_matches}")
print(f"  + Multi-matches: {num_multi_matches}")
print(f"  + Non-matches  : {num_non_matches}")
print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_FMOS.original_count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

zlim_deep = [0, 3]
zlim = [0, 4]

value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_FMOS'] > 0.0) & (catalog_UVISTA.redshift['z_FMOS_flag'] < 8)
fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_FMOS.catalog} vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_FMOS'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
ax.set_xlabel(f"$z_{{peak}}$ [{catalog_UVISTA.catalog}]")
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_FMOS.catalog}]")
ax.set_xlim(zlim_deep)
ax.set_ylim(zlim_deep)

fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_FMOS.catalog}")
value_filter = (catalog_UVISTA.redshift['z_ZCB'] > 0.0) & (catalog_UVISTA.redshift['z_ZCB_flag'] < 8) & (catalog_UVISTA.redshift['z_FMOS'] > 0.0) & (catalog_UVISTA.redshift['z_FMOS_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCB'][value_filter], catalog_UVISTA.redshift['z_FMOS'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
value_filter = (catalog_UVISTA.redshift['z_ZCD'] > 0.0) & (catalog_UVISTA.redshift['z_ZCD_flag'] < 8) & (catalog_UVISTA.redshift['z_FMOS'] > 0.0) & (catalog_UVISTA.redshift['z_FMOS_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCD'][value_filter], catalog_UVISTA.redshift['z_FMOS'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
value_filter = (catalog_UVISTA.redshift['z_3DHST'] > 0.0) & (catalog_UVISTA.redshift['z_3DHST_flag'] < 8) & (catalog_UVISTA.redshift['z_FMOS'] > 0.0) & (catalog_UVISTA.redshift['z_FMOS_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_3DHST'][value_filter], catalog_UVISTA.redshift['z_FMOS'][value_filter], label=f"3DHST ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour4)
value_filter = (catalog_UVISTA.redshift['z_VUDS'] > 0.0) & (catalog_UVISTA.redshift['z_VUDS_flag'] < 8) & (catalog_UVISTA.redshift['z_FMOS'] > 0.0) & (catalog_UVISTA.redshift['z_FMOS_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_VUDS'][value_filter], catalog_UVISTA.redshift['z_FMOS'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
value_filter = (catalog_UVISTA.redshift['z_Casey'] > 0.0) & (catalog_UVISTA.redshift['z_Casey_flag'] < 8) & (catalog_UVISTA.redshift['z_FMOS'] > 0.0) & (catalog_UVISTA.redshift['z_FMOS_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_Casey'][value_filter], catalog_UVISTA.redshift['z_FMOS'][value_filter], label=f"Casey ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
value_filter = (catalog_UVISTA.redshift['z_DEIMOS'] > 0.0) & (catalog_UVISTA.redshift['z_DEIMOS_flag'] < 8) & (catalog_UVISTA.redshift['z_FMOS'] > 0.0) & (catalog_UVISTA.redshift['z_FMOS_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_DEIMOS'][value_filter], catalog_UVISTA.redshift['z_FMOS'][value_filter], label=f"DEIMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour5)
value_filter = (catalog_UVISTA.redshift['z_MOSDEF'] > 0.0) & (catalog_UVISTA.redshift['z_MOSDEF_flag'] < 8) & (catalog_UVISTA.redshift['z_FMOS'] > 0.0) & (catalog_UVISTA.redshift['z_FMOS_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_MOSDEF'][value_filter], catalog_UVISTA.redshift['z_FMOS'][value_filter], label=f"MOSDEF ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour4)
ax.set_xlabel('$z_{spec}$ [Other]')
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_FMOS.catalog}]")
ax.set_xlim(zlim)
ax.set_ylim(zlim)
ax.legend(loc='lower right')

del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_FMOS' in locals():
    catalog.close(catalog_FMOS)
    del catalog_FMOS, matches_FMOS, catalog_params_FMOS

### Cross-Match KMOS3D (2019-07-12)

In [ ]:
catalog_params_KMOS3D = catalog.get_params('KMOS3D', 'COSMOS')
catalog_KMOS3D = catalog.read(catalog_params_KMOS3D)
print(f"{catalog_KMOS3D.catalog}: {catalog_KMOS3D.count} sources")

In [ ]:
matches_KMOS3D = match.cross_match_catalogs_by_ra_dec(catalog_UVISTA, catalog_KMOS3D)
print(f"Number of Matches: {matches_KMOS3D.count}")

In [ ]:
match.append_cross_matched_data(matches_KMOS3D, catalog_UVISTA, catalog_KMOS3D)

In [ ]:
value_filter   = (catalog_UVISTA.idmap['id']  < 1e7) & (catalog_UVISTA.idmap['KMOS3D'] >= 0)
other_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['KMOS3D'] >= 0) & (catalog_UVISTA.idmap['KMOS3D_dist'] > 0.0)
multi_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['KMOS3D'] >= 0) & (catalog_UVISTA.idmap['KMOS3D_dist'] == 0.0) & (catalog_UVISTA.idmap['KMOS3D_num'] > 1)
dup_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['KMOS3D'] >= 0) & (catalog_UVISTA.idmap['KMOS3D_dist'] == 0.0) & (catalog_UVISTA.idmap['KMOS3D_num'] == 1)
non_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['KMOS3D'] >= 0) & (catalog_UVISTA.idmap['KMOS3D_dist'] == 0.0) & (catalog_UVISTA.idmap['KMOS3D_num'] == 0)

num_matches = np.sum(value_filter)
num_other_matches = np.sum(other_match_filter)
num_multi_matches = np.sum(multi_match_filter)
num_dup_matches = np.sum(dup_match_filter)
num_non_matches = np.sum(non_match_filter)
print(f"Number of {catalog_KMOS3D.name} Redshifts:")
print(f"    Matches      : {num_matches} (diff={matches_KMOS3D.count - num_matches - num_other_matches})")
print(f"  + Other matches: {num_other_matches}")
print(f"  + Dup-matches  : {num_dup_matches}")
print(f"  + Multi-matches: {num_multi_matches}")
print(f"  + Non-matches  : {num_non_matches}")
print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_KMOS3D.original_count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

zlim_deep = [0, 4]
zlim = [0, 3.5]

value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_KMOS3D'] > 0.0) & (catalog_UVISTA.redshift['z_KMOS3D_flag'] < 8)
fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_KMOS3D.catalog} vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_KMOS3D'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
ax.set_xlabel(f"$z_{{peak}}$ [{catalog_UVISTA.catalog}]")
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_KMOS3D.catalog}]")
ax.set_xlim(zlim_deep)
ax.set_ylim(zlim_deep)

fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_KMOS3D.catalog}")
value_filter = (catalog_UVISTA.redshift['z_ZCB'] > 0.0) & (catalog_UVISTA.redshift['z_ZCB_flag'] < 8) & (catalog_UVISTA.redshift['z_KMOS3D'] > 0.0) & (catalog_UVISTA.redshift['z_KMOS3D_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCB'][value_filter], catalog_UVISTA.redshift['z_KMOS3D'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
value_filter = (catalog_UVISTA.redshift['z_ZCD'] > 0.0) & (catalog_UVISTA.redshift['z_ZCD_flag'] < 8) & (catalog_UVISTA.redshift['z_KMOS3D'] > 0.0) & (catalog_UVISTA.redshift['z_KMOS3D_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCD'][value_filter], catalog_UVISTA.redshift['z_KMOS3D'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
value_filter = (catalog_UVISTA.redshift['z_3DHST'] > 0.0) & (catalog_UVISTA.redshift['z_3DHST_flag'] < 8) & (catalog_UVISTA.redshift['z_KMOS3D'] > 0.0) & (catalog_UVISTA.redshift['z_KMOS3D_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_3DHST'][value_filter], catalog_UVISTA.redshift['z_KMOS3D'][value_filter], label=f"3DHST ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour4)
value_filter = (catalog_UVISTA.redshift['z_VUDS'] > 0.0) & (catalog_UVISTA.redshift['z_VUDS_flag'] < 8) & (catalog_UVISTA.redshift['z_KMOS3D'] > 0.0) & (catalog_UVISTA.redshift['z_KMOS3D_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_VUDS'][value_filter], catalog_UVISTA.redshift['z_KMOS3D'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
value_filter = (catalog_UVISTA.redshift['z_Casey'] > 0.0) & (catalog_UVISTA.redshift['z_Casey_flag'] < 8) & (catalog_UVISTA.redshift['z_KMOS3D'] > 0.0) & (catalog_UVISTA.redshift['z_KMOS3D_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_Casey'][value_filter], catalog_UVISTA.redshift['z_KMOS3D'][value_filter], label=f"Casey ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
value_filter = (catalog_UVISTA.redshift['z_DEIMOS'] > 0.0) & (catalog_UVISTA.redshift['z_DEIMOS_flag'] < 8) & (catalog_UVISTA.redshift['z_KMOS3D'] > 0.0) & (catalog_UVISTA.redshift['z_KMOS3D_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_DEIMOS'][value_filter], catalog_UVISTA.redshift['z_KMOS3D'][value_filter], label=f"DEIMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour5)
value_filter = (catalog_UVISTA.redshift['z_MOSDEF'] > 0.0) & (catalog_UVISTA.redshift['z_MOSDEF_flag'] < 8) & (catalog_UVISTA.redshift['z_KMOS3D'] > 0.0) & (catalog_UVISTA.redshift['z_KMOS3D_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_MOSDEF'][value_filter], catalog_UVISTA.redshift['z_KMOS3D'][value_filter], label=f"MOSDEF ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour4)
value_filter = (catalog_UVISTA.redshift['z_FMOS'] > 0.0) & (catalog_UVISTA.redshift['z_FMOS_flag'] < 8) & (catalog_UVISTA.redshift['z_KMOS3D'] > 0.0) & (catalog_UVISTA.redshift['z_KMOS3D_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_FMOS'][value_filter], catalog_UVISTA.redshift['z_KMOS3D'][value_filter], label=f"FMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour3)
ax.set_xlabel('$z_{spec}$ [Other]')
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_KMOS3D.catalog}]")
ax.set_xlim(zlim)
ax.set_ylim(zlim)
ax.legend(loc='lower right')

del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_KMOS3D' in locals():
    catalog.close(catalog_KMOS3D)
    del catalog_KMOS3D, matches_KMOS3D, catalog_params_KMOS3D

### Cross-Match C3R2 (DR1+DR2, 2019-04-11 + DR3, 2021-06-18)

In [ ]:
catalog_params_C3R2 = catalog.get_params('C3R2', 'COSMOS')
catalog_C3R2 = catalog.read(catalog_params_C3R2)
print(f"{catalog_C3R2.catalog}: {catalog_C3R2.count} sources")

In [ ]:
matches_C3R2 = match.cross_match_catalogs_by_ra_dec(catalog_UVISTA, catalog_C3R2)
print(f"Number of Matches: {matches_C3R2.count}")

In [ ]:
match.append_cross_matched_data(matches_C3R2, catalog_UVISTA, catalog_C3R2)

In [ ]:
value_filter   = (catalog_UVISTA.idmap['id']  < 1e7) & (catalog_UVISTA.idmap['C3R2'] >= 0)
other_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['C3R2'] >= 0) & (catalog_UVISTA.idmap['C3R2_dist'] > 0.0)
multi_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['C3R2'] >= 0) & (catalog_UVISTA.idmap['C3R2_dist'] == 0.0) & (catalog_UVISTA.idmap['C3R2_num'] > 1)
dup_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['C3R2'] >= 0) & (catalog_UVISTA.idmap['C3R2_dist'] == 0.0) & (catalog_UVISTA.idmap['C3R2_num'] == 1)
non_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['C3R2'] >= 0) & (catalog_UVISTA.idmap['C3R2_dist'] == 0.0) & (catalog_UVISTA.idmap['C3R2_num'] == 0)

num_matches = np.sum(value_filter)
num_other_matches = np.sum(other_match_filter)
num_multi_matches = np.sum(multi_match_filter)
num_dup_matches = np.sum(dup_match_filter)
num_non_matches = np.sum(non_match_filter)
print(f"Number of {catalog_C3R2.name} Redshifts:")
print(f"    Matches      : {num_matches} (diff={matches_C3R2.count - num_matches - num_other_matches})")
print(f"  + Other matches: {num_other_matches}")
print(f"  + Dup-matches  : {num_dup_matches}")
print(f"  + Multi-matches: {num_multi_matches}")
print(f"  + Non-matches  : {num_non_matches}")
print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_C3R2.original_count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

zlim_deep = [0, 4]
zlim = [0, 3.5]

value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_C3R2'] > 0.0) & (catalog_UVISTA.redshift['z_C3R2_flag'] < 8)
fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_C3R2.catalog} vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_C3R2'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
ax.set_xlabel(f"$z_{{peak}}$ [{catalog_UVISTA.catalog}]")
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_C3R2.catalog}]")
ax.set_xlim(zlim_deep)
ax.set_ylim(zlim_deep)

fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_C3R2.catalog}")
value_filter = (catalog_UVISTA.redshift['z_ZCB'] > 0.0) & (catalog_UVISTA.redshift['z_ZCB_flag'] < 8) & (catalog_UVISTA.redshift['z_C3R2'] > 0.0) & (catalog_UVISTA.redshift['z_C3R2_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCB'][value_filter], catalog_UVISTA.redshift['z_C3R2'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
value_filter = (catalog_UVISTA.redshift['z_ZCD'] > 0.0) & (catalog_UVISTA.redshift['z_ZCD_flag'] < 8) & (catalog_UVISTA.redshift['z_C3R2'] > 0.0) & (catalog_UVISTA.redshift['z_C3R2_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCD'][value_filter], catalog_UVISTA.redshift['z_C3R2'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
value_filter = (catalog_UVISTA.redshift['z_3DHST'] > 0.0) & (catalog_UVISTA.redshift['z_3DHST_flag'] < 8) & (catalog_UVISTA.redshift['z_C3R2'] > 0.0) & (catalog_UVISTA.redshift['z_C3R2_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_3DHST'][value_filter], catalog_UVISTA.redshift['z_C3R2'][value_filter], label=f"3DHST ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour4)
value_filter = (catalog_UVISTA.redshift['z_VUDS'] > 0.0) & (catalog_UVISTA.redshift['z_VUDS_flag'] < 8) & (catalog_UVISTA.redshift['z_C3R2'] > 0.0) & (catalog_UVISTA.redshift['z_C3R2_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_VUDS'][value_filter], catalog_UVISTA.redshift['z_C3R2'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
value_filter = (catalog_UVISTA.redshift['z_Casey'] > 0.0) & (catalog_UVISTA.redshift['z_Casey_flag'] < 8) & (catalog_UVISTA.redshift['z_C3R2'] > 0.0) & (catalog_UVISTA.redshift['z_C3R2_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_Casey'][value_filter], catalog_UVISTA.redshift['z_C3R2'][value_filter], label=f"Casey ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
value_filter = (catalog_UVISTA.redshift['z_DEIMOS'] > 0.0) & (catalog_UVISTA.redshift['z_DEIMOS_flag'] < 8) & (catalog_UVISTA.redshift['z_C3R2'] > 0.0) & (catalog_UVISTA.redshift['z_C3R2_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_DEIMOS'][value_filter], catalog_UVISTA.redshift['z_C3R2'][value_filter], label=f"DEIMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour5)
value_filter = (catalog_UVISTA.redshift['z_MOSDEF'] > 0.0) & (catalog_UVISTA.redshift['z_MOSDEF_flag'] < 8) & (catalog_UVISTA.redshift['z_C3R2'] > 0.0) & (catalog_UVISTA.redshift['z_C3R2_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_MOSDEF'][value_filter], catalog_UVISTA.redshift['z_C3R2'][value_filter], label=f"MOSDEF ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour4)
value_filter = (catalog_UVISTA.redshift['z_FMOS'] > 0.0) & (catalog_UVISTA.redshift['z_FMOS_flag'] < 8) & (catalog_UVISTA.redshift['z_C3R2'] > 0.0) & (catalog_UVISTA.redshift['z_C3R2_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_FMOS'][value_filter], catalog_UVISTA.redshift['z_C3R2'][value_filter], label=f"FMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour3)
value_filter = (catalog_UVISTA.redshift['z_KMOS3D'] > 0.0) & (catalog_UVISTA.redshift['z_KMOS3D_flag'] < 8) & (catalog_UVISTA.redshift['z_C3R2'] > 0.0) & (catalog_UVISTA.redshift['z_C3R2_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_KMOS3D'][value_filter], catalog_UVISTA.redshift['z_C3R2'][value_filter], label=f"KMOS3D ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour2)
ax.set_xlabel('$z_{spec}$ [Other]')
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_C3R2.catalog}]")
ax.set_xlim(zlim)
ax.set_ylim(zlim)
ax.legend(loc='lower right')

del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_C3R2' in locals():
    catalog.close(catalog_C3R2)
    del catalog_C3R2, matches_C3R2, catalog_params_C3R2

### Cross-Match LEGA-C (DR3, 2021-08-02)

In [ ]:
catalog_params_LEGAC = catalog.get_params('LEGAC')
catalog_LEGAC = catalog.read(catalog_params_LEGAC)
print(f"{catalog_LEGAC.catalog}: {catalog_LEGAC.count} sources")

In [ ]:
matches_LEGAC = match.cross_match_catalogs_by_id_ra_dec(catalog_UVISTA, catalog_LEGAC, id_field2='ID')
print(f"Number of Matches: {matches_LEGAC.count}")

In [ ]:
match.append_cross_matched_data(matches_LEGAC, catalog_UVISTA, catalog_LEGAC)

In [ ]:
value_filter   = (catalog_UVISTA.idmap['id'] < 1e7) & (catalog_UVISTA.idmap['LEGAC'] >= 0)
other_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['LEGAC'] >= 0) & (catalog_UVISTA.idmap['LEGAC_dist'] > 0.0)
multi_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['LEGAC'] >= 0) & (catalog_UVISTA.idmap['LEGAC_dist'] == 0.0) & (catalog_UVISTA.idmap['LEGAC_num'] > 1)
dup_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['LEGAC'] >= 0) & (catalog_UVISTA.idmap['LEGAC_dist'] == 0.0) & (catalog_UVISTA.idmap['LEGAC_num'] == 1)
non_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['LEGAC'] >= 0) & (catalog_UVISTA.idmap['LEGAC_dist'] == 0.0) & (catalog_UVISTA.idmap['LEGAC_num'] == 0)

num_matches = np.sum(value_filter)
num_other_matches = np.sum(other_match_filter)
num_multi_matches = np.sum(multi_match_filter)
num_dup_matches = np.sum(dup_match_filter)
num_non_matches = np.sum(non_match_filter)
print(f"Number of {catalog_LEGAC.name} Redshifts:")
print(f"    Matches      : {num_matches} (diff={matches_LEGAC.count - num_matches - num_other_matches})")
print(f"  + Other matches: {num_other_matches}")
print(f"  + Dup-matches  : {num_dup_matches}")
print(f"  + Multi-matches: {num_multi_matches}")
print(f"  + Non-matches  : {num_non_matches}")
print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_LEGAC.original_count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

zlim_deep = [0, 3]
zlim = [0, 2]

value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_LEGAC'] > 0.0) & (catalog_UVISTA.redshift['z_LEGAC_flag'] < 8)
fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_LEGAC.catalog} vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_LEGAC'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
ax.set_xlabel(f"$z_{{peak}}$ [{catalog_UVISTA.catalog}]")
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_LEGAC.catalog}]")
ax.set_xlim(zlim_deep)
ax.set_ylim(zlim_deep)

fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_LEGAC.catalog}")
value_filter = (catalog_UVISTA.redshift['z_ZCB'] > 0.0) & (catalog_UVISTA.redshift['z_ZCB_flag'] < 8) & (catalog_UVISTA.redshift['z_LEGAC'] > 0.0) & (catalog_UVISTA.redshift['z_LEGAC_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCB'][value_filter], catalog_UVISTA.redshift['z_LEGAC'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
value_filter = (catalog_UVISTA.redshift['z_ZCD'] > 0.0) & (catalog_UVISTA.redshift['z_ZCD_flag'] < 8) & (catalog_UVISTA.redshift['z_LEGAC'] > 0.0) & (catalog_UVISTA.redshift['z_LEGAC_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCD'][value_filter], catalog_UVISTA.redshift['z_LEGAC'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
value_filter = (catalog_UVISTA.redshift['z_3DHST'] > 0.0) & (catalog_UVISTA.redshift['z_3DHST_flag'] < 8) & (catalog_UVISTA.redshift['z_LEGAC'] > 0.0) & (catalog_UVISTA.redshift['z_LEGAC_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_3DHST'][value_filter], catalog_UVISTA.redshift['z_LEGAC'][value_filter], label=f"3DHST ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour4)
value_filter = (catalog_UVISTA.redshift['z_VUDS'] > 0.0) & (catalog_UVISTA.redshift['z_VUDS_flag'] < 8) & (catalog_UVISTA.redshift['z_LEGAC'] > 0.0) & (catalog_UVISTA.redshift['z_LEGAC_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_VUDS'][value_filter], catalog_UVISTA.redshift['z_LEGAC'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
value_filter = (catalog_UVISTA.redshift['z_Casey'] > 0.0) & (catalog_UVISTA.redshift['z_Casey_flag'] < 8) & (catalog_UVISTA.redshift['z_LEGAC'] > 0.0) & (catalog_UVISTA.redshift['z_LEGAC_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_Casey'][value_filter], catalog_UVISTA.redshift['z_LEGAC'][value_filter], label=f"Casey ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
value_filter = (catalog_UVISTA.redshift['z_DEIMOS'] > 0.0) & (catalog_UVISTA.redshift['z_DEIMOS_flag'] < 8) & (catalog_UVISTA.redshift['z_LEGAC'] > 0.0) & (catalog_UVISTA.redshift['z_LEGAC_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_DEIMOS'][value_filter], catalog_UVISTA.redshift['z_LEGAC'][value_filter], label=f"DEIMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour5)
value_filter = (catalog_UVISTA.redshift['z_MOSDEF'] > 0.0) & (catalog_UVISTA.redshift['z_MOSDEF_flag'] < 8) & (catalog_UVISTA.redshift['z_LEGAC'] > 0.0) & (catalog_UVISTA.redshift['z_LEGAC_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_MOSDEF'][value_filter], catalog_UVISTA.redshift['z_LEGAC'][value_filter], label=f"MOSDEF ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour4)
value_filter = (catalog_UVISTA.redshift['z_FMOS'] > 0.0) & (catalog_UVISTA.redshift['z_FMOS_flag'] < 8) & (catalog_UVISTA.redshift['z_LEGAC'] > 0.0) & (catalog_UVISTA.redshift['z_LEGAC_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_FMOS'][value_filter], catalog_UVISTA.redshift['z_LEGAC'][value_filter], label=f"FMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour3)
value_filter = (catalog_UVISTA.redshift['z_KMOS3D'] > 0.0) & (catalog_UVISTA.redshift['z_KMOS3D_flag'] < 8) & (catalog_UVISTA.redshift['z_LEGAC'] > 0.0) & (catalog_UVISTA.redshift['z_LEGAC_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_KMOS3D'][value_filter], catalog_UVISTA.redshift['z_LEGAC'][value_filter], label=f"KMOS3D ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour2)
value_filter = (catalog_UVISTA.redshift['z_C3R2'] > 0.0) & (catalog_UVISTA.redshift['z_C3R2_flag'] < 8) & (catalog_UVISTA.redshift['z_LEGAC'] > 0.0) & (catalog_UVISTA.redshift['z_LEGAC_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_C3R2'][value_filter], catalog_UVISTA.redshift['z_LEGAC'][value_filter], label=f"C3R2 ({np.sum(value_filter)})", s=5, marker='^', edgecolors='none', facecolors=plot_options.colour3)
ax.set_xlabel('$z_{spec}$ [Other]')
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_LEGAC.catalog}]")
ax.set_xlim(zlim)
ax.set_ylim(zlim)
ax.legend(loc='lower right')

del zlim_deep, zlim, fig, ax

In [ ]:
if not keep_open and 'catalog_LEGAC' in locals():
    catalog.close(catalog_LEGAC)
    del catalog_LEGAC, matches_LEGAC, catalog_params_LEGAC

### Cross-Match HELP z-Spec Catalog (v2.8 Public)

In [ ]:
catalog_params_HELP = catalog.get_params('HELP', 'COSMOS')
catalog_HELP = catalog.read(catalog_params_HELP)
print(f"{catalog_HELP.catalog}: {catalog_HELP.count} sources")

In [ ]:
matches_HELP = match.cross_match_catalogs_by_ra_dec(catalog_UVISTA, catalog_HELP)
print(f"Number of Matches: {matches_HELP.count}")

In [ ]:
match.append_cross_matched_data(matches_HELP, catalog_UVISTA, catalog_HELP)

In [ ]:
value_filter   = (catalog_UVISTA.idmap['id']  < 1e7) & (catalog_UVISTA.idmap['HELP'] >= 0)
other_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['HELP'] >= 0) & (catalog_UVISTA.idmap['HELP_dist'] > 0.0)
multi_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['HELP'] >= 0) & (catalog_UVISTA.idmap['HELP_dist'] == 0.0) & (catalog_UVISTA.idmap['HELP_num'] > 1)
dup_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['HELP'] >= 0) & (catalog_UVISTA.idmap['HELP_dist'] == 0.0) & (catalog_UVISTA.idmap['HELP_num'] == 1)
non_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['HELP'] >= 0) & (catalog_UVISTA.idmap['HELP_dist'] == 0.0) & (catalog_UVISTA.idmap['HELP_num'] == 0)

num_matches = np.sum(value_filter)
num_other_matches = np.sum(other_match_filter)
num_multi_matches = np.sum(multi_match_filter)
num_dup_matches = np.sum(dup_match_filter)
num_non_matches = np.sum(non_match_filter)
print(f"Number of {catalog_HELP.name} Redshifts:")
print(f"    Matches      : {num_matches} (diff={matches_HELP.count - num_matches - num_other_matches})")
print(f"  + Other matches: {num_other_matches}")
print(f"  + Dup-matches  : {num_dup_matches}")
print(f"  + Multi-matches: {num_multi_matches}")
print(f"  + Non-matches  : {num_non_matches}")
print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_HELP.original_count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

zlim_deep = [0, 6]
zlim = [0, 6]

value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_HELP'] > 0.0) & (catalog_UVISTA.redshift['z_HELP_flag'] < 8)
fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_HELP.catalog} vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_HELP'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
ax.set_xlabel(f"$z_{{peak}}$ [{catalog_UVISTA.catalog}]")
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_HELP.catalog}]")
ax.set_xlim(zlim_deep)
ax.set_ylim(zlim_deep)

fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_HELP.catalog}")
value_filter = (catalog_UVISTA.redshift['z_ZCB'] > 0.0) & (catalog_UVISTA.redshift['z_ZCB_flag'] < 8) & (catalog_UVISTA.redshift['z_HELP'] > 0.0) & (catalog_UVISTA.redshift['z_HELP_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCB'][value_filter], catalog_UVISTA.redshift['z_HELP'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
value_filter = (catalog_UVISTA.redshift['z_ZCD'] > 0.0) & (catalog_UVISTA.redshift['z_ZCD_flag'] < 8) & (catalog_UVISTA.redshift['z_HELP'] > 0.0) & (catalog_UVISTA.redshift['z_HELP_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCD'][value_filter], catalog_UVISTA.redshift['z_HELP'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
value_filter = (catalog_UVISTA.redshift['z_3DHST'] > 0.0) & (catalog_UVISTA.redshift['z_3DHST_flag'] < 8) & (catalog_UVISTA.redshift['z_HELP'] > 0.0) & (catalog_UVISTA.redshift['z_HELP_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_3DHST'][value_filter], catalog_UVISTA.redshift['z_HELP'][value_filter], label=f"3DHST ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour4)
value_filter = (catalog_UVISTA.redshift['z_VUDS'] > 0.0) & (catalog_UVISTA.redshift['z_VUDS_flag'] < 8) & (catalog_UVISTA.redshift['z_HELP'] > 0.0) & (catalog_UVISTA.redshift['z_HELP_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_VUDS'][value_filter], catalog_UVISTA.redshift['z_HELP'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
value_filter = (catalog_UVISTA.redshift['z_Casey'] > 0.0) & (catalog_UVISTA.redshift['z_Casey_flag'] < 8) & (catalog_UVISTA.redshift['z_HELP'] > 0.0) & (catalog_UVISTA.redshift['z_HELP_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_Casey'][value_filter], catalog_UVISTA.redshift['z_HELP'][value_filter], label=f"Casey ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
value_filter = (catalog_UVISTA.redshift['z_DEIMOS'] > 0.0) & (catalog_UVISTA.redshift['z_DEIMOS_flag'] < 8) & (catalog_UVISTA.redshift['z_HELP'] > 0.0) & (catalog_UVISTA.redshift['z_HELP_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_DEIMOS'][value_filter], catalog_UVISTA.redshift['z_HELP'][value_filter], label=f"DEIMOS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
value_filter = (catalog_UVISTA.redshift['z_MOSDEF'] > 0.0) & (catalog_UVISTA.redshift['z_MOSDEF_flag'] < 8) & (catalog_UVISTA.redshift['z_HELP'] > 0.0) & (catalog_UVISTA.redshift['z_HELP_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_MOSDEF'][value_filter], catalog_UVISTA.redshift['z_HELP'][value_filter], label=f"MOSDEF ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour4)
value_filter = (catalog_UVISTA.redshift['z_FMOS'] > 0.0) & (catalog_UVISTA.redshift['z_FMOS_flag'] < 8) & (catalog_UVISTA.redshift['z_HELP'] > 0.0) & (catalog_UVISTA.redshift['z_HELP_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_FMOS'][value_filter], catalog_UVISTA.redshift['z_HELP'][value_filter], label=f"FMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour3)
value_filter = (catalog_UVISTA.redshift['z_KMOS3D'] > 0.0) & (catalog_UVISTA.redshift['z_KMOS3D_flag'] < 8) & (catalog_UVISTA.redshift['z_HELP'] > 0.0) & (catalog_UVISTA.redshift['z_HELP_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_KMOS3D'][value_filter], catalog_UVISTA.redshift['z_HELP'][value_filter], label=f"KMOS3D ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour2)
value_filter = (catalog_UVISTA.redshift['z_C3R2'] > 0.0) & (catalog_UVISTA.redshift['z_C3R2_flag'] < 8) & (catalog_UVISTA.redshift['z_HELP'] > 0.0) & (catalog_UVISTA.redshift['z_HELP_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_C3R2'][value_filter], catalog_UVISTA.redshift['z_HELP'][value_filter], label=f"C3R2 ({np.sum(value_filter)})", s=5, marker='^', edgecolors='none', facecolors=plot_options.colour3)
value_filter = (catalog_UVISTA.redshift['z_LEGAC'] > 0.0) & (catalog_UVISTA.redshift['z_LEGAC_flag'] < 8) & (catalog_UVISTA.redshift['z_HELP'] > 0.0) & (catalog_UVISTA.redshift['z_HELP_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_LEGAC'][value_filter], catalog_UVISTA.redshift['z_HELP'][value_filter], label=f"LEGAC ({np.sum(value_filter)})", s=5, marker='^', edgecolors='none', facecolors=plot_options.colour4)
ax.set_xlabel('$z_{spec}$ [Other]')
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_HELP.catalog}]")
ax.set_xlim(zlim)
ax.set_ylim(zlim)
ax.legend(loc='lower right')

del zlim_deep, zlim, fig, ax

In [ ]:
# TODO: Check matches based on catalog_HELP.sources['Z_SOURCE']

In [ ]:
if not keep_open and 'catalog_HELP' in locals():
    catalog.close(catalog_HELP)
    del catalog_HELP, matches_HELP, catalog_params_HELP

### Cross-Match DESI (EDR, 2023-06-09)

In [ ]:
catalog_params_DESI = catalog.get_params('DESI', 'COSMOS')
catalog_DESI = catalog.read(catalog_params_DESI, region=catalog_region_UVISTA, region_wcs=catalog_params_UVISTA.wcs)
print(f"{catalog_DESI.catalog}: {catalog_DESI.count} sources")

In [ ]:
matches_DESI = match.cross_match_catalogs_by_ra_dec(catalog_UVISTA, catalog_DESI)
print(f"Number of Matches: {matches_DESI.count}")

In [ ]:
match.append_cross_matched_data(matches_DESI, catalog_UVISTA, catalog_DESI)

In [ ]:
value_filter   = (catalog_UVISTA.idmap['id']  < 1e7) & (catalog_UVISTA.idmap['DESI'] >= 0)
other_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['DESI'] >= 0) & (catalog_UVISTA.idmap['DESI_dist'] > 0.0)
multi_match_filter = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['DESI'] >= 0) & (catalog_UVISTA.idmap['DESI_dist'] == 0.0) & (catalog_UVISTA.idmap['DESI_num'] > 1)
dup_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['DESI'] >= 0) & (catalog_UVISTA.idmap['DESI_dist'] == 0.0) & (catalog_UVISTA.idmap['DESI_num'] == 1)
non_match_filter   = (catalog_UVISTA.idmap['id'] >= 1e7) & (catalog_UVISTA.idmap['DESI'] >= 0) & (catalog_UVISTA.idmap['DESI_dist'] == 0.0) & (catalog_UVISTA.idmap['DESI_num'] == 0)

num_matches = np.sum(value_filter)
num_other_matches = np.sum(other_match_filter)
num_multi_matches = np.sum(multi_match_filter)
num_dup_matches = np.sum(dup_match_filter)
num_non_matches = np.sum(non_match_filter)
print(f"Number of {catalog_DESI.name} Redshifts:")
print(f"    Matches      : {num_matches} (diff={matches_DESI.count - num_matches - num_other_matches})")
print(f"  + Other matches: {num_other_matches}")
print(f"  + Dup-matches  : {num_dup_matches}")
print(f"  + Multi-matches: {num_multi_matches}")
print(f"  + Non-matches  : {num_non_matches}")
print(f"  = Total        : {num_matches + num_other_matches + num_dup_matches + num_multi_matches + num_non_matches} (diff={catalog_DESI.original_count - num_matches - num_other_matches - num_dup_matches - num_multi_matches - num_non_matches})")

zlim_deep = [0, 3.5]
zlim = [0, 2.0]

value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_DESI'] > 0.0) & (catalog_UVISTA.redshift['z_DESI_flag'] < 8)
fig, ax = plot.create_plot(plot_mode, title=f"Match Sources: {catalog_DESI.catalog} vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_DESI'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
ax.set_xlabel(f"$z_{{peak}}$ [{catalog_UVISTA.catalog}]")
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_DESI.catalog}]")
ax.set_xlim(zlim_deep)
ax.set_ylim(zlim_deep)

fig, ax = plot.create_plot(plot_mode, title=f"Compare {catalog_DESI.catalog}")
value_filter = (catalog_UVISTA.redshift['z_ZCB'] > 0.0) & (catalog_UVISTA.redshift['z_ZCB_flag'] < 8) & (catalog_UVISTA.redshift['z_DESI'] > 0.0) & (catalog_UVISTA.redshift['z_DESI_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCB'][value_filter], catalog_UVISTA.redshift['z_DESI'][value_filter], label=f"ZCB ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
value_filter = (catalog_UVISTA.redshift['z_ZCD'] > 0.0) & (catalog_UVISTA.redshift['z_ZCD_flag'] < 8) & (catalog_UVISTA.redshift['z_DESI'] > 0.0) & (catalog_UVISTA.redshift['z_DESI_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_ZCD'][value_filter], catalog_UVISTA.redshift['z_DESI'][value_filter], label=f"ZCD ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3)
value_filter = (catalog_UVISTA.redshift['z_3DHST'] > 0.0) & (catalog_UVISTA.redshift['z_3DHST_flag'] < 8) & (catalog_UVISTA.redshift['z_DESI'] > 0.0) & (catalog_UVISTA.redshift['z_DESI_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_3DHST'][value_filter], catalog_UVISTA.redshift['z_DESI'][value_filter], label=f"3DHST ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour4)
value_filter = (catalog_UVISTA.redshift['z_VUDS'] > 0.0) & (catalog_UVISTA.redshift['z_VUDS_flag'] < 8) & (catalog_UVISTA.redshift['z_DESI'] > 0.0) & (catalog_UVISTA.redshift['z_DESI_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_VUDS'][value_filter], catalog_UVISTA.redshift['z_DESI'][value_filter], label=f"VUDS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
value_filter = (catalog_UVISTA.redshift['z_Casey'] > 0.0) & (catalog_UVISTA.redshift['z_Casey_flag'] < 8) & (catalog_UVISTA.redshift['z_DESI'] > 0.0) & (catalog_UVISTA.redshift['z_DESI_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_Casey'][value_filter], catalog_UVISTA.redshift['z_DESI'][value_filter], label=f"Casey ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6)
value_filter = (catalog_UVISTA.redshift['z_DEIMOS'] > 0.0) & (catalog_UVISTA.redshift['z_DEIMOS_flag'] < 8) & (catalog_UVISTA.redshift['z_DESI'] > 0.0) & (catalog_UVISTA.redshift['z_DESI_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_DEIMOS'][value_filter], catalog_UVISTA.redshift['z_DESI'][value_filter], label=f"DEIMOS ({np.sum(value_filter)})", s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5)
value_filter = (catalog_UVISTA.redshift['z_MOSDEF'] > 0.0) & (catalog_UVISTA.redshift['z_MOSDEF_flag'] < 8) & (catalog_UVISTA.redshift['z_DESI'] > 0.0) & (catalog_UVISTA.redshift['z_DESI_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_MOSDEF'][value_filter], catalog_UVISTA.redshift['z_DESI'][value_filter], label=f"MOSDEF ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour4)
value_filter = (catalog_UVISTA.redshift['z_FMOS'] > 0.0) & (catalog_UVISTA.redshift['z_FMOS_flag'] < 8) & (catalog_UVISTA.redshift['z_DESI'] > 0.0) & (catalog_UVISTA.redshift['z_DESI_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_FMOS'][value_filter], catalog_UVISTA.redshift['z_DESI'][value_filter], label=f"FMOS ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour3)
value_filter = (catalog_UVISTA.redshift['z_KMOS3D'] > 0.0) & (catalog_UVISTA.redshift['z_KMOS3D_flag'] < 8) & (catalog_UVISTA.redshift['z_DESI'] > 0.0) & (catalog_UVISTA.redshift['z_DESI_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_KMOS3D'][value_filter], catalog_UVISTA.redshift['z_DESI'][value_filter], label=f"KMOS3D ({np.sum(value_filter)})", s=5, marker='s', edgecolors='none', facecolors=plot_options.colour2)
value_filter = (catalog_UVISTA.redshift['z_C3R2'] > 0.0) & (catalog_UVISTA.redshift['z_C3R2_flag'] < 8) & (catalog_UVISTA.redshift['z_DESI'] > 0.0) & (catalog_UVISTA.redshift['z_DESI_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_C3R2'][value_filter], catalog_UVISTA.redshift['z_DESI'][value_filter], label=f"C3R2 ({np.sum(value_filter)})", s=5, marker='^', edgecolors='none', facecolors=plot_options.colour3)
value_filter = (catalog_UVISTA.redshift['z_LEGAC'] > 0.0) & (catalog_UVISTA.redshift['z_LEGAC_flag'] < 8) & (catalog_UVISTA.redshift['z_DESI'] > 0.0) & (catalog_UVISTA.redshift['z_DESI_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_LEGAC'][value_filter], catalog_UVISTA.redshift['z_DESI'][value_filter], label=f"LEGAC ({np.sum(value_filter)})", s=5, marker='^', edgecolors='none', facecolors=plot_options.colour4)
value_filter = (catalog_UVISTA.redshift['z_HELP'] > 0.0) & (catalog_UVISTA.redshift['z_HELP_flag'] < 8) & (catalog_UVISTA.redshift['z_DESI'] > 0.0) & (catalog_UVISTA.redshift['z_DESI_flag'] < 8)
ax.scatter(catalog_UVISTA.redshift['z_HELP'][value_filter], catalog_UVISTA.redshift['z_DESI'][value_filter], label=f"HELP ({np.sum(value_filter)})", s=5, marker='^', edgecolors='none', facecolors=plot_options.colour5)
ax.set_xlabel('$z_{spec}$ [Other]')
ax.set_ylabel(f"$z_{{spec}}$ [{catalog_DESI.catalog}]")
ax.set_xlim(zlim)
ax.set_ylim(zlim)
ax.legend(loc='lower right')

del zlim_deep, zlim, fig, ax

In [ ]:
z = catalog_DESI.sources['Z']
dz = catalog_DESI.sources['ZERR']
rdz = dz / (1 + z)

fig, ax = plot.create_plot(plot_mode, title=f"DESI Redshift Error, N={len(z)}")
ax.scatter(z, rdz, s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
ax.set_xlabel('$z_{{spec}}$')
ax.set_ylabel('$\sigma_z / (1+z)$')

fig, ax = plot.create_plot(plot_mode, title=f"DESI Redshift Error, N={len(z)}")
ax.hist(rdz*1e4, int(np.sqrt(len(rdz))), rwidth=0.95, color=plot_options.hist_colour)
ax.set_xlabel('$\sigma_z / (1+z) \\times 10^{-4}$')
ax.set_ylabel('Galaxies')
ax.set_xlim([0, 2])

del z, dz, rdz, fig, ax

In [ ]:
if not keep_open and 'catalog_DESI' in locals():
    catalog.close(catalog_DESI)
    del catalog_DESI, matches_DESI, catalog_params_DESI

### Consolidate UltraVISTA Catalog

In [ ]:
catalog.consolidate_best_data(catalog_UVISTA)

### Summary

In [ ]:
value_filter = (catalog_UVISTA.sources['star'] == 0) & (catalog_UVISTA.best['z_best'] > 0.0) & (catalog_UVISTA.best['z_best_flag'] < 10)
sources, source_counts = np.unique(catalog_UVISTA.best['z_best_source'][value_filter], return_counts=True)
source_ticks = np.arange(len(sources))
fig, ax = plot.create_plot(plot_mode, title=f"All $z_{{spec}}$, N={np.sum(value_filter)}")
b = ax.bar(source_ticks, source_counts, color=plot_options.hist_colour)
ax.bar_label(b, label_type='center', color=plot_options.hist_label_color, fontsize=10, rotation=90)
ax.set_xticks(source_ticks)
ax.set_xticklabels(sources, rotation=45, ha='right')
ax.set_xlabel('Source')
ax.set_ylabel('Galaxies')

print(f"All z-Specs: {np.sum(value_filter)}")

value_filter = (catalog_UVISTA.sources['star'] == 0) & (catalog_UVISTA.best['z_best'] > 0.0) & (catalog_UVISTA.best['z_best_flag'] < 8)
sources, source_counts = np.unique(catalog_UVISTA.best['z_best_source'][value_filter], return_counts=True)
source_ticks = np.arange(len(sources))
fig, ax = plot.create_plot(plot_mode, title=f"Usable $z_{{spec}}$, N={np.sum(value_filter)}")
b = ax.bar(source_ticks, source_counts, color=plot_options.hist_colour)
ax.bar_label(b, label_type='center', color=plot_options.hist_label_color, fontsize=10, rotation=90)
ax.set_xticks(source_ticks)
ax.set_xticklabels(sources, rotation=45, ha='right')
ax.set_xlabel('Source')
ax.set_ylabel('Galaxies')

print(f"Usable z-Specs: {np.sum(value_filter)}")

value_filter = (catalog_UVISTA.sources['star'] == 0) & (catalog_UVISTA.best['z_best'] > 0.0) & (catalog_UVISTA.best['z_best_flag'] <= 2)
sources, source_counts = np.unique(catalog_UVISTA.best['z_best_source'][value_filter], return_counts=True)
source_ticks = np.arange(len(sources))
fig, ax = plot.create_plot(plot_mode, title=f"Best $z_{{spec}}$, N={np.sum(value_filter)}")
b = ax.bar(source_ticks, source_counts, color=plot_options.hist_colour)
ax.bar_label(b, label_type='center', color=plot_options.hist_label_color, fontsize=10, rotation=90)
ax.set_xticks(source_ticks)
ax.set_xticklabels(sources, rotation=45, ha='right')
ax.set_xlabel('Source')
ax.set_ylabel('Galaxies')

print(f"Best z-Specs: {np.sum(value_filter)}")

value_filter = (catalog_UVISTA.sources['star'] == 0) & (catalog_UVISTA.best['z_best'] > 0.0) & (catalog_UVISTA.best['z_best_flag'] <= 2) & (catalog_UVISTA.best['id'] < 1e7)
sources, source_counts = np.unique(catalog_UVISTA.best['z_best_source'][value_filter], return_counts=True)
source_ticks = np.arange(len(sources))
fig, ax = plot.create_plot(plot_mode, title=f"Best Matched $z_{{spec}}$, N={np.sum(value_filter)}")
b = ax.bar(source_ticks, source_counts, color=plot_options.hist_colour)
ax.bar_label(b, label_type='center', color=plot_options.hist_label_color, fontsize=10, rotation=90)
ax.set_xticks(source_ticks)
ax.set_xticklabels(sources, rotation=45, ha='right')
ax.set_xlabel('Source')
ax.set_ylabel('Galaxies')

print(f"Best Matched z-Specs: {np.sum(value_filter)}")

zlim = [0, 6]
fig, ax = plot.create_plot(plot_mode, title=f"Best Matched $z_{{spec}}$, N={np.sum(value_filter)}")
value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_ZCB_flag'] <= 2)
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_ZCB'][value_filter], label='ZCB', s=5, marker='o', edgecolors='none', facecolors=plot_options.colour2)
value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_ZCD_flag'] <= 2)
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_ZCD'][value_filter], label='ZCD', s=5, marker='o', edgecolors='none', facecolors=plot_options.colour3, alpha=0.6)
value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_3DHST_flag'] <= 2)
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_3DHST'][value_filter], label='3DHST', s=5, marker='o', edgecolors='none', facecolors=plot_options.colour4, alpha=0.6)
value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_VUDS_flag'] <= 2)
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_VUDS'][value_filter], label='VUDS', s=5, marker='o', edgecolors='none', facecolors=plot_options.colour5, alpha=0.6)
value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_Casey_flag'] <= 2)
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_Casey'][value_filter], label='Casey', s=5, marker='o', edgecolors='none', facecolors=plot_options.colour6, alpha=0.6)
value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_DEIMOS_flag'] <= 2)
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_DEIMOS'][value_filter], label='DEIMOS', s=5, marker='s', edgecolors='none', facecolors=plot_options.colour5, alpha=0.6)
value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_MOSDEF_flag'] <= 2)
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_MOSDEF'][value_filter], label='MOSDEF', s=5, marker='s', edgecolors='none', facecolors=plot_options.colour4, alpha=0.6)
value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_FMOS_flag'] <= 2)
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_FMOS'][value_filter], label='FMOS', s=5, marker='s', edgecolors='none', facecolors=plot_options.colour3, alpha=0.6)
value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_KMOS3D_flag'] <= 2)
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_KMOS3D'][value_filter], label='KMOS3D', s=5, marker='s', edgecolors='none', facecolors=plot_options.colour2, alpha=0.6)
value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_C3R2_flag'] <= 2)
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_C3R2'][value_filter], label='C3R2', s=5, marker='^', edgecolors='none', facecolors=plot_options.colour3, alpha=0.6)
value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_LEGAC_flag'] <= 2)
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_LEGAC'][value_filter], label='LEGAC', s=5, marker='^', edgecolors='none', facecolors=plot_options.colour4, alpha=0.6)
value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_HELP_flag'] <= 2)
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_HELP'][value_filter], label='HELP', s=5, marker='^', edgecolors='none', facecolors=plot_options.colour5, alpha=0.6)
value_filter = (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.redshift['z_DESI_flag'] <= 2)
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.redshift['z_DESI'][value_filter], label='DESI', s=5, marker='^', edgecolors='none', facecolors=plot_options.colour6, alpha=0.6)
ax.set_xlabel(f"$z_{{peak}}$ [{catalog_UVISTA.catalog}]")
ax.set_ylabel(f"$z_{{spec}}$")
ax.set_xlim(zlim)
ax.set_ylim(zlim)
ax.legend(loc='lower right')

del fig, ax, value_filter


In [ ]:
zlim = [0, 6]

value_filter = (catalog_UVISTA.sources['star'] == 0) & (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.best['z_best'] > 0.0) & (catalog_UVISTA.best['z_best_flag'] <= 2)

fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (Best) vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.best['z_best'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
ax.set_xlabel('$z_{peak}$')
ax.set_ylabel('$z_{best}$')
ax.set_xlim(zlim)
ax.set_ylim(zlim)

sources, source_counts = np.unique(catalog_UVISTA.best['z_best_source'][value_filter], return_counts=True)
source_ticks = np.arange(len(sources))
fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (Best), N={np.sum(value_filter)}")
b = ax.bar(source_ticks, source_counts, color=plot_options.hist_colour)
ax.bar_label(b, label_type='center', color=plot_options.hist_label_color, fontsize=10, rotation=90)
ax.set_xticks(source_ticks)
ax.set_xticklabels(sources, rotation=45, ha='right')
ax.set_xlabel('Source')
ax.set_ylabel('Galaxies')

value_filter = (catalog_UVISTA.sources['star'] == 0) & (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.best['z_best'] > 0.0) & (catalog_UVISTA.best['z_best_flag'] > 2) & (catalog_UVISTA.best['z_best_flag'] < 8)
if np.sum(value_filter) > 0:
    fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (Other) vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.best['z_best'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel('$z_{peak}$')
    ax.set_ylabel('$z_{best}$')
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)

    sources, source_counts = np.unique(catalog_UVISTA.best['z_best_source'][value_filter], return_counts=True)
    source_ticks = np.arange(len(sources))
    fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (Other), N={np.sum(value_filter)}")
    b = ax.bar(source_ticks, source_counts, color=plot_options.hist_colour)
    ax.bar_label(b, label_type='center', color=plot_options.hist_label_color, fontsize=10, rotation=90)
    ax.set_xticks(source_ticks, sources)
    ax.set_xticklabels(sources, rotation=45, ha='right')
    ax.set_xlabel('Source')
    ax.set_ylabel('Galaxies')

value_filter = (catalog_UVISTA.sources['star'] == 0) & (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.best['z_best'] > 0.0) & (catalog_UVISTA.best['z_best_flag'] == 8)
if np.sum(value_filter) > 0:
    fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (Inconsistent) vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.best['z_best'][value_filter], label='Inconsistent', s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel('$z_{peak}$')
    ax.set_ylabel('$z_{best}$')
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)

    sources, source_counts = np.unique(catalog_UVISTA.best['z_best_source'][value_filter], return_counts=True)
    source_ticks = np.arange(len(sources))
    fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (Inconsistent), N={np.sum(value_filter)}")
    b = ax.bar(source_ticks, source_counts, color=plot_options.hist_colour)
    ax.bar_label(b, label_type='center', color=plot_options.hist_label_color, fontsize=10, rotation=90)
    ax.set_xticks(source_ticks, sources)
    ax.set_xticklabels(sources, rotation=45, ha='right')
    ax.set_xlabel('Source')
    ax.set_ylabel('Galaxies')

value_filter = (catalog_UVISTA.sources['star'] == 0) & (catalog_UVISTA.redshift['z_peak'] > 0.0) & (catalog_UVISTA.best['z_best'] > 0.0) & (catalog_UVISTA.best['z_best_flag'] == 9)
if np.sum(value_filter) > 0:
    fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (Unreliable) vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.best['z_best'][value_filter], label='Unreliable', s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel('$z_{peak}$')
    ax.set_ylabel('$z_{best}$')
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)

    sources, source_counts = np.unique(catalog_UVISTA.best['z_best_source'][value_filter], return_counts=True)
    source_ticks = np.arange(len(sources))
    fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (Unreliable), N={np.sum(value_filter)}")
    b = ax.bar(source_ticks, source_counts, color=plot_options.hist_colour)
    ax.bar_label(b, label_type='center', color=plot_options.hist_label_color, fontsize=10, rotation=90)
    ax.set_xticks(source_ticks, sources)
    ax.set_xticklabels(sources, rotation=45, ha='right')
    ax.set_xlabel('Source')
    ax.set_ylabel('Galaxies')

value_filter = (catalog_UVISTA.sources['star'] == 0) & (catalog_UVISTA.redshift['z_peak'] > 0.0) & ((catalog_UVISTA.best['z_best'] == 0.0) | (catalog_UVISTA.best['z_best_flag'] == 99))
if np.sum(value_filter) > 0:
    fig, ax = plot.create_plot(plot_mode, title=f"$z_{{best}}$ (None) vs {catalog_UVISTA.catalog}, N={np.sum(value_filter)}")
    ax.scatter(catalog_UVISTA.redshift['z_peak'][value_filter], catalog_UVISTA.best['z_best'][value_filter], s=5, marker='o', edgecolors='none', facecolors=plot_options.fit_data_colour)
    ax.set_xlabel('$z_{peak}$')
    ax.set_ylabel('$z_{best}$')
    ax.set_xlim(zlim)
    ax.set_ylim(zlim)

del zlim, fig, ax, b, value_filter

### Compare Emission Line Flux

In [ ]:
line_names = ['Ha', 'OIIIb', 'Hb', 'OIIb']
line_labels = ['Ha+[NII]', '[OIII]', 'Hb', '[OII]']
line_other_names = [['NIIa', 'NIIb'], ['OIIIa'], [], ['OIIa']]
line_sources = ['3DHST', 'MOSDEF', 'FMOS', 'LEGAC', 'DESI']
line_markers = ['o'    , 's'     , '^'   , 'v'    , 'p'   ]
line_sizes =   [15     , 15      , 25    , 25     , 25    ]
line_colours = plot_options.colours[2:7]

fluxLim = [1e-1, 1e3]

for i in np.arange(len(line_names)):
    fig, ax = plot.create_plot(plot_mode, title=f"Compare $f_{{{line_labels[i]}}}$")

    for j in np.arange(len(line_sources)):
        total_flux_j = np.zeros((len(catalog_UVISTA.lines)))
        value_filter = catalog_UVISTA.lines[f"flag_{line_names[i]}_{line_sources[j]}"] < 9
        total_flux_j[value_filter] = catalog_UVISTA.lines[f"f_{line_names[i]}_{line_sources[j]}"][value_filter]

        for l in np.arange(len(line_other_names[i])):
            value_filter = catalog_UVISTA.lines[f"flag_{line_other_names[i][l]}_{line_sources[j]}"] < 9
            total_flux_j[value_filter] += catalog_UVISTA.lines[f"f_{line_other_names[i][l]}_{line_sources[j]}"][value_filter]

        for k in np.arange(j+1, len(line_sources)):
            total_flux_k = np.zeros((len(catalog_UVISTA.lines)))
            value_filter = catalog_UVISTA.lines[f"flag_{line_names[i]}_{line_sources[k]}"] < 9
            total_flux_k[value_filter] = catalog_UVISTA.lines[f"f_{line_names[i]}_{line_sources[k]}"][value_filter]

            for l in np.arange(len(line_other_names[i])):
                value_filter = catalog_UVISTA.lines[f"flag_{line_other_names[i][l]}_{line_sources[k]}"] < 9
                total_flux_k[value_filter] += catalog_UVISTA.lines[f"f_{line_other_names[i][l]}_{line_sources[k]}"][value_filter]

            value_filter = (total_flux_j > 0) & (total_flux_k > 0)

            if np.sum(value_filter) == 0:
                continue

            ax.scatter(total_flux_k[value_filter], total_flux_j[value_filter], label=f"{line_sources[j]} vs {line_sources[k]} ({np.sum(value_filter)})", s=line_sizes[k], marker=line_markers[k], facecolors=line_colours[j], edgecolors='k', linewidth=0.25)

    ax.set_xlabel(f"$f_{{{line_labels[i]}}}$")
    ax.set_ylabel(f"$f_{{{line_labels[i]}}}$")
    ax.plot(fluxLim, fluxLim, linestyle='-', color=plot_options.colour7)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlim(fluxLim)
    ax.set_ylim(fluxLim)
    ax.legend(loc='lower right')

del line_sources, line_names, line_other_names, line_labels, line_markers, line_sizes, line_colours, i, j, k, l, fluxLim, fig, ax


### Save Cross-Matched UltraVISTA Catalog

In [ ]:
catalog_params_UVISTAPLUS = catalog.get_params('UVISTA-PLUS')

if do_save:
    catalog.save_matched_catalog(catalog_UVISTA, catalog_params_UVISTAPLUS, save_format='pickel', new_name='UVISTA-PLUS')

if do_save_ascii:
    catalog.save_matched_catalog(catalog_UVISTA, catalog_params_UVISTAPLUS, save_format='ascii', new_name='UVISTA-PLUS')

### Close Catalog

In [ ]:
if not keep_open and 'catalog_UVISTA' in locals():
    catalog.close(catalog_UVISTA)
    del catalog_UVISTA, catalog_params_UVISTA, catalog_params_UVISTAPLUS